In [5]:
from tqdm import tqdm

bigfilepath = "../data/pageviews-20210101-automated"
smallfilepath = "../exploration/sample.txt"
outputfilepath = "../data/output_test"

In [31]:
outstring = ""
max_iterations = 10 * 1000 * 1000

code_to_hour_dic = {letter:str(num) for num,letter in enumerate("ABCDEFGHIJKLMNOPQRSTUVWXYZ")}

with open(bigfilepath) as infile:
    for i, line in tqdm(enumerate(infile)):
        # split line into list of words
        wordlist = line.split(" ")

        if len(wordlist) == 6:
            # the case where we have an ID
            if wordlist[2].isnumeric() or wordlist[2] == "null":
                # the case where a numeric value (or null) is at third position
                # so:
                    # title: abcd, id:1234
                    # title: abcd, id:null
                    # title: 1234, id:1234
                    # title: 1234, id:null
                pass
            else:
                # case where there is a word at third position that is neither numeric nor null
                # so:
                    # title: 1234, id:abcd --> swap
                    # title: abcd, id:abcd --> ???
                    # title: null, id:abcd --> swap
                # we swap, if the value at 2nd position is either null or numeric
                if wordlist[1].isalpha() or (wordlist[1] == "null"):
                    wordlist[1], wordlist[2] = wordlist[2], wordlist[1]
        else:
            # the case where we do not have an ID
            wordlist.insert(2, "null")

        # at this point there is for sure* an id or null at position 3
        # *unless there is an entry that has 6 rows with pos 2 and 3 both non-null words
        if wordlist[2] == "null":
            wordlist[2] = "-1"

        # now we divide to days
        wordlists = []

        # divide last string into separate entries
        count = ""
        time = ""
        first = True
        for symbol in wordlist[-1]:
            if symbol.isalpha():
                if not first:
                    # if this is not the first iteration, count is a valid number and time has also been set
                    # since its not the first, we just encountered a letter again, hence we write to the wordlists
                    # we write the original list, without the A1B2C3 string and instead encoding the first pair A1 to time: 0, count: 1
                    wordlists.append(wordlist[:-1] + [time, count])
                    count = ""
                else: first = False  # if it was the first, the next one wont be the first
                # in any case we get the time                    
                time = code_to_hour_dic[symbol]
            elif symbol.isnumeric():
                count += symbol
            elif symbol == "\n":
                # we arrived at the end
                wordlists.append(wordlist[:-1] + [time, count])
                break
        # now we have a wordlists which contains the counts split by hour

        # recombine cleaned list of words into line
        for wordlist in wordlists:
            outstring += " ".join(wordlist) + "\n"
        if i > max_iterations:
            break

with open(outputfilepath, "w") as outfile:
    outfile.write(outstring)

8353208it [00:35, 237287.16it/s]


In [28]:
!head -n10 "../data/output_test"

aa.wikibooks Main_Page -1 desktop 8 5 8
aa.wikibooks Special:Book -1 desktop 1 5 1
aa.wikibooks Special:CiteThisPage -1 desktop 1 5 1
aa.wikibooks Special:DownloadAsPdf -1 desktop 1 5 1
aa.wikibooks Special:SpecialPages -1 desktop 1 5 1
aa.wikibooks Special:UserLogin -1 desktop 1 5 1
aa.wikipedia Special:Statistics -1 desktop 24 0 1
aa.wikipedia Special:Statistics -1 desktop 24 1 1
aa.wikipedia Special:Statistics -1 desktop 24 2 1
aa.wikipedia Special:Statistics -1 desktop 24 3 1


In [46]:
# !head -n10 "../exploration/sample.txt" && echo "" && head -n10 "../data/output_test"